In [3]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
## Langsmith tracking
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['USER_AGENT'] = "human"

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader('https://docs.smith.langchain.com/administration/tutorials/manage_spend')
loader

In [6]:
docs = loader.load()

In [ ]:
docs
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4o")
llm

In [9]:
# Load data -> docs -> Divide into chunks -> text -> vectors -> Embeddings -> storage
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = splitter.split_documents(docs)

In [12]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [14]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(splits, embeddings)

In [ ]:
db

In [ ]:
query = "What does the first metric track?"
result = db.similarity_search(query)
print(result[0].page_content)

In [ ]:
## Retrieval chain and document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    """
)

chain = create_stuff_documents_chain(llm, prompt)
chain

In [ ]:
from langchain_core.documents import Document
chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

In [30]:
### Retriever
retriever = db.as_retriever()

In [ ]:
from langchain.chains import create_retrieval_chain
r_chain = create_retrieval_chain(retriever, chain)
r_chain

In [ ]:
response = r_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']